# Auto-Proving LEAN Statements

This notebook attempts to automatically prove the translated LEAN 4 statements using LEAN's built-in tactics and automation.

## Why Use Auto-Provers?

Instead of manually proving each theorem, we leverage LEAN 4's powerful **automated reasoning** capabilities:

- **`aesop`** (Automated Extensible Search for Obvious Proofs) is LEAN's state-of-the-art automation tool that can handle:
  - Propositional and first-order logic
  - Inductive reasoning
  - Simplification and rewriting
  - Case analysis and splitting

This notebook offers two approaches:
1. **Simple (Recommended)**: Just use `aesop` - fast and effective for most cases
2. **Comprehensive**: Try multiple tactics (`aesop`, `simp_all`, `omega`, `decide`, etc.) to analyze which work best

For most use cases, `aesop` alone is sufficient and much faster!

## Setup and Imports

In [19]:
import json
import subprocess
import os
import tempfile
from pathlib import Path
from typing import Dict, List, Tuple
import re

## Configuration

In [20]:
# Input file with translated LEAN statements
INPUT_FILE = "test_file/output_namespacePropsScript_lean.json"

# Output file to save proof results
OUTPUT_FILE = "test_file/proof_results.json"

# LEAN project directory (will be created if it doesn't exist)
LEAN_PROJECT_DIR = "lean_proof_project"

## Check LEAN Installation

## Inspect Translated Statements

Let's first check what the translated LEAN statements look like to identify any issues:

In [21]:
# Load and inspect the translated statements
try:
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        translated_data = json.load(f)
    
    print(f"Found {len(translated_data)} translated statements\n")
    print("="*80)
    print("SAMPLE TRANSLATED STATEMENTS:")
    print("="*80)
    
    # Show first 3 statements
    for i, item in enumerate(translated_data[:3]):
        print(f"\n{i+1}. {item['kind']}: {item['name']}")
        print(f"   Original HOL4: {item.get('original_hol4', 'N/A')}")
        print(f"   LEAN Translation: {item['statement']}")
        print("-"*80)
    
    # Check for common issues
    print("\n" + "="*80)
    print("DIAGNOSTICS:")
    print("="*80)
    
    # Check if statements use undefined types
    undefined_types = set()
    for item in translated_data:
        stmt = item['statement']
        # Look for common undefined type patterns
        if 'Id' in stmt and 'mk_id' in stmt:
            undefined_types.add('Id (custom type)')
        if 'nsLookup' in stmt:
            undefined_types.add('nsLookup (custom function)')
        if 'SOME' in stmt or 'NONE' in stmt:
            print(f"⚠ Warning: Found HOL4 syntax (SOME/NONE) in {item['name']}")
    
    if undefined_types:
        print(f"\n⚠ Found references to potentially undefined types/functions:")
        for t in undefined_types:
            print(f"  - {t}")
        print("\n💡 These need to be defined in the LEAN file before the theorems can be checked.")
    
except FileNotFoundError:
    print(f"❌ File not found: {INPUT_FILE}")
    print("Please run the translation notebook first!")
except json.JSONDecodeError as e:
    print(f"❌ Error reading JSON: {e}")
except Exception as e:
    print(f"❌ Error: {e}")

Found 56 translated statements

SAMPLE TRANSLATED STATEMENTS:

1. Theorem: mk_id_surj
   Original HOL4: !id. ?p n. id = mk_id p n
   LEAN Translation: theorem mk_id_surj {IdType PType NType : Type} (mk_id : PType → NType → IdType) : ∀ (id : IdType), ∃ (p : PType) (n : NType), id = mk_id p n
--------------------------------------------------------------------------------

2. Theorem: mk_id_thm
   Original HOL4: !id. mk_id (id_to_mods id) (id_to_n id) = id
   LEAN Translation: theorem mk_id_thm {IdType ModsType NType : Type} (mk_id : ModsType → NType → IdType) (id_to_mods : IdType → ModsType) (id_to_n : IdType → NType) : ∀ (id : IdType), mk_id (id_to_mods id) (id_to_n id) = id
--------------------------------------------------------------------------------

3. Theorem: nsSub_mono2
   Original HOL4: (!x y z. nsLookup e1 x = SOME y ∧ nsLookup e2 x = SOME z ∧ R1 x y z ⇒ R2 x y z) ⇒ (nsSub R1 e1 e2 ⇒ nsSub R2 e1 e2)
   LEAN Translation: theorem nsSub_mono2 {α β : Type} (e1 e2 : α → Option β)

### Fix Translation Issues (Optional)

If your translations have the `theorem` keyword in them, you can fix them here:

In [22]:
# Optional: Fix the translated file by removing duplicate 'theorem' keywords
import re

def fix_translations(input_file: str, output_file: str = None):
    """Remove 'theorem' keywords from translated statements."""
    if output_file is None:
        output_file = input_file  # Overwrite the original
    
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    fixed_count = 0
    for item in data:
        original_stmt = item['statement']
        
        # Remove 'theorem name :' pattern
        cleaned = re.sub(r'^\s*theorem\s+\w+\s*:', '', original_stmt).strip()
        
        # Also try to remove just the theorem keyword
        if cleaned == original_stmt:
            cleaned = re.sub(r'^\s*theorem\s+', '', original_stmt).strip()
        
        if cleaned != original_stmt:
            item['statement'] = cleaned
            fixed_count += 1
    
    # Save fixed data
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    
    print(f"✓ Fixed {fixed_count} statements")
    print(f"✓ Saved to: {output_file}")
    
    return fixed_count

# Uncomment the line below to fix your translation file:
# fix_translations(INPUT_FILE)

## Test Single Statement First

Before processing all statements, let's test one to see what errors we get and fix them:

In [33]:
# Test with the first statement
if not os.path.exists(INPUT_FILE):
    print(f"❌ File not found: {INPUT_FILE}")
    print("Please run the translation notebook first!")
else:
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    if not data:
        print("❌ No statements found in the file!")
    else:
        # Get the first statement
        test_item = data[0]
        
        print("="*80)
        print("TESTING FIRST STATEMENT")
        print("="*80)
        print(f"\nTheorem: {test_item['name']}")
        print(f"Kind: {test_item['kind']}")
        print(f"\nOriginal HOL4:")
        print(f"  {test_item.get('original_hol4', 'N/A')}")
        print(f"\nTranslated LEAN:")
        print(f"  {test_item['statement']}")
        
        # Create LEAN project if needed
        if not create_lean_project(LEAN_PROJECT_DIR):
            print("\n❌ Failed to create LEAN project")
        else:
            print("\n" + "-"*80)
            print("ATTEMPTING PROOF...")
            print("-"*80)
            
            # Attempt to prove
            result = attempt_simple_proof(
                test_item['statement'],
                test_item['name'],
                test_item['kind'],
                LEAN_PROJECT_DIR
            )
            
            print(f"\nResult: {'✓ PROVED' if result['proved'] else '✗ FAILED'}")
            
            if result.get('cleaned_statement') != test_item['statement']:
                print(f"\nCleaned statement:")
                print(f"  {result['cleaned_statement']}")
            
            print(f"\nLEAN file created: {result['lean_file']}")
            
            if not result['proved']:
                print("\n" + "="*80)
                print("ERROR OUTPUT:")
                print("="*80)
                print(result['lean_output'])
                print("\n" + "="*80)
                print("NEXT STEPS TO FIX:")
                print("="*80)
                
                # Analyze the error
                error_output = result['lean_output'].lower()
                
                if 'unknown identifier' in error_output or 'unknown constant' in error_output:
                    print("❌ Missing type/function definitions")
                    print("   → Need to define custom types used in the theorem")
                    print("   → Check what types are referenced (Id, nsLookup, etc.)")
                    
                if 'unexpected token' in error_output:
                    print("❌ Syntax error in the statement")
                    print("   → The translation might not be valid LEAN 4 syntax")
                    print("   → Consider re-translating with a better prompt")
                    
                if 'import' in error_output or 'module' in error_output:
                    print("❌ Import/module issues")
                    print("   → Mathlib might not be installed")
                    print("   → Run: lake exe cache get")
                
                if 'type mismatch' in error_output:
                    print("❌ Type mismatch")
                    print("   → The types in the translation don't match LEAN's expectations")
                    print("   → The translation needs refinement")
            else:
                print("\n✓ SUCCESS! This statement can be automatically proved.")
                print("  You can now run the full proof attempt on all statements.")

TESTING FIRST STATEMENT

Theorem: mk_id_surj
Kind: Theorem

Original HOL4:
  !id. ?p n. id = mk_id p n

Translated LEAN:
  theorem mk_id_surj {IdType PType NType : Type} (mk_id : PType → NType → IdType) : ∀ (id : IdType), ∃ (p : PType) (n : NType), id = mk_id p n
Project directory 'lean_proof_project' already exists.

--------------------------------------------------------------------------------
ATTEMPTING PROOF...
--------------------------------------------------------------------------------

Result: ✗ FAILED

Cleaned statement:
  mk_id_surj {IdType PType NType : Type} (mk_id : PType → NType → IdType) : ∀ (id : IdType), ∃ (p : PType) (n : NType), id = mk_id p n

LEAN file created: lean_proof_project\ProofProject\mk_id_surj.lean

ERROR OUTPUT:
lean_proof_project\ProofProject\mk_id_surj.lean:4:0: error: unknown module prefix 'Mathlib'

No directory 'Mathlib' or file 'Mathlib.olean' in the search path entries:
c:\Users\my-pc\.elan\toolchains\leanprover--lean4---v4.24.0\lib\lean


NEX

### View Generated LEAN File

Let's see what the actual LEAN file looks like:

In [ ]:
# View the generated LEAN file
lean_files = list(Path(LEAN_PROJECT_DIR).glob("ProofProject/*.lean"))

if lean_files:
    # Show the most recently created file
    latest_file = max(lean_files, key=lambda p: p.stat().st_mtime)
    
    print(f"Generated LEAN file: {latest_file}")
    print("="*80)
    print("CONTENT:")
    print("="*80)
    
    with open(latest_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    print(content)
else:
    print("No LEAN files generated yet. Run the test cell above first.")

### Test a Specific Statement by Index

Try different statements to see which ones work:

In [ ]:
# Test a specific statement by index
TEST_INDEX = 0  # Change this to test different statements (0-based index)

if os.path.exists(INPUT_FILE):
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    if TEST_INDEX < len(data):
        test_item = data[TEST_INDEX]
        
        print(f"Testing statement {TEST_INDEX + 1} of {len(data)}")
        print(f"Name: {test_item['name']}\n")
        
        # Create LEAN project if needed
        create_lean_project(LEAN_PROJECT_DIR)
        
        # Attempt to prove
        result = attempt_simple_proof(
            test_item['statement'],
            test_item['name'],
            test_item['kind'],
            LEAN_PROJECT_DIR
        )
        
        print(f"Result: {'✓ PROVED' if result['proved'] else '✗ FAILED'}")
        
        if not result['proved']:
            print(f"\nError output (first 500 chars):")
            print(result['lean_output'][:500])
            print("\n...")
    else:
        print(f"❌ Index {TEST_INDEX} out of range. File has {len(data)} statements.")
else:
    print(f"❌ File not found: {INPUT_FILE}")

### Common Fixes

Based on the errors above, here are some common fixes you can try:

In [ ]:
# Common fixes to try

print("COMMON FIXES FOR LEAN ERRORS:")
print("="*80)

print("\n1. If you see 'unknown identifier' or 'unknown constant':")
print("   → The statement uses undefined types/functions")
print("   → Solution: Re-translate with simpler, more generic types")
print("   → Or: Define the types manually in a separate LEAN file")

print("\n2. If you see 'unexpected token theorem':")
print("   → The translation included 'theorem' keyword")
print("   → Solution: Already handled by clean_lean_statement()")
print("   → Run: fix_translations(INPUT_FILE) to fix the JSON file")

print("\n3. If you see 'failed to synthesize instance':")
print("   → LEAN can't infer the type class instance")
print("   → Solution: The translation needs more explicit type annotations")

print("\n4. If you see 'import not found' or Mathlib errors:")
print("   → Mathlib is not installed")
print("   → Solution: In the LEAN project dir, run:")
print("     cd " + LEAN_PROJECT_DIR)
print("     lake exe cache get")
print("     lake build")

print("\n5. If the statement is too complex for auto-proving:")
print("   → aesop can't find a proof automatically")
print("   → Solution: These need manual proof development")
print("   → Consider: Simplifying the statement or adding lemmas")

print("\n" + "="*80)
print("\nRECOMMENDED WORKFLOW:")
print("="*80)
print("1. Run the diagnostic cell to see the translations")
print("2. Run fix_translations() if needed")
print("3. Test one statement at a time")
print("4. If errors persist, re-translate with a better prompt that:")
print("   - Uses only LEAN standard library types")
print("   - Avoids custom types from HOL4")
print("   - Makes statements more generic/abstract")
print("   - Does NOT include the 'theorem' keyword")

### Understanding "unexpected token 'theorem'" Error

This error typically means:

1. **Duplicate 'theorem' keyword**: The translation included `theorem name :` in the statement, but our code adds it again
   - ✅ **FIXED**: Added automatic cleaning to remove duplicate keywords
   
2. **Missing Type Definitions**: The LEAN translation references types (like `Id`, `nsLookup`, etc.) that aren't defined
   - ⚠️ **NOTE**: Custom types from your HOL4 code need to be defined in LEAN
   
3. **Missing Imports**: Required libraries aren't imported
   - ✅ **FIXED**: Now importing Mathlib.Tactic and common libraries

**What We've Fixed:**
- Automatically remove `theorem` keyword from statements if present
- Add proper Mathlib imports to all generated files
- Show cleaned statements in output for debugging

Run the diagnostic cell above to see if your translations have these issues!

In [23]:
def check_lean_installation() -> bool:
    """
    Check if LEAN 4 is installed and accessible.
    """
    try:
        result = subprocess.run(
            ['lean', '--version'],
            capture_output=True,
            text=True,
            timeout=10
        )
        if result.returncode == 0:
            print(f"LEAN found: {result.stdout.strip()}")
            return True
        else:
            print("LEAN not found or not working properly.")
            return False
    except FileNotFoundError:
        print("LEAN 4 is not installed or not in PATH.")
        print("Please install LEAN 4 from: https://leanprover.github.io/lean4/doc/quickstart.html")
        return False
    except Exception as e:
        print(f"Error checking LEAN installation: {e}")
        return False

# Check if LEAN is installed
lean_available = check_lean_installation()

LEAN found: Lean (version 4.24.0, x86_64-w64-windows-gnu, commit 797c613eb9b6d4ec95db23e3e00af9ac6657f24b, Release)


## LEAN Project Setup Functions

In [24]:
def create_lean_project(project_dir: str) -> bool:
    """
    Create a basic LEAN 4 project structure.
    """
    try:
        project_path = Path(project_dir)
        
        if project_path.exists():
            print(f"Project directory '{project_dir}' already exists.")
            return True
        
        # Create project directory
        project_path.mkdir(parents=True, exist_ok=True)
        
        # Create lakefile.lean
        lakefile_content = """import Lake
open Lake DSL

package «ProofProject» where
  -- add package configuration options here

lean_lib «ProofProject» where
  -- add library configuration options here
"""
        (project_path / "lakefile.lean").write_text(lakefile_content)
        
        # Create ProofProject directory
        (project_path / "ProofProject").mkdir(exist_ok=True)
        
        print(f"Created LEAN project at '{project_dir}'")
        return True
        
    except Exception as e:
        print(f"Error creating LEAN project: {e}")
        return False

## Proof Generation Functions

## About LEAN's Automated Provers

LEAN 4 has powerful automated proof tactics:

1. **`aesop`** - The most powerful general-purpose automation tool. It combines multiple strategies including term rewriting, case splitting, and forward/backward reasoning.

2. **`simp_all` + `omega`** - Combines aggressive simplification with linear arithmetic solving. Great for numeric properties.

3. **`decide`** - For decidable propositions (boolean expressions, finite types, etc.)

4. **`tauto`** - For propositional logic tautologies

5. **`trivial`** - For trivial proofs by definition

The functions below will try these tactics systematically to find which ones can automatically prove each theorem.

In [25]:
def generate_lean_proof_file(statement: str, name: str, kind: str) -> str:
    """
    Generate a LEAN file with the theorem using LEAN's auto-provers.
    """
    # Clean the statement first - remove any 'theorem' keywords
    import re
    statement_clean = re.sub(r'^\s*theorem\s+\w+\s*:', '', statement).strip()
    if statement_clean == statement:
        statement_clean = re.sub(r'^\s*theorem\s+', '', statement).strip()
    statement_clean = statement_clean.strip()
    
    # Use LEAN's most powerful automated tactics
    lean_content = f"""-- Theorem: {name}
-- Original HOL4 {kind}

import Mathlib.Tactic
import Mathlib.Data.List.Basic
import Mathlib.Data.Option.Basic

-- Attempt 1: Using aesop (most powerful general-purpose automation)
theorem {name}_v1 : {statement_clean} := by
  aesop

-- Attempt 2: Using simp with all lemmas and omega
theorem {name}_v2 : {statement_clean} := by
  simp_all
  omega

-- Attempt 3: Using decide (for decidable propositions)
theorem {name}_v3 : {statement_clean} := by
  decide

-- Attempt 4: Using tauto (for propositional logic tautologies)
theorem {name}_v4 : {statement_clean} := by
  tauto

-- Attempt 5: Using simp with omega
theorem {name}_v5 : {statement_clean} := by
  simp
  omega

-- Attempt 6: Using trivial (for trivial proofs)
theorem {name}_v6 : {statement_clean} := by
  trivial

-- Attempt 7: Using rfl (for definitional equality)
theorem {name}_v7 : {statement_clean} := by
  rfl

-- Attempt 8: Combination: intro, simp_all, aesop
theorem {name}_v8 : {statement_clean} := by
  intro
  simp_all
  aesop

-- Attempt 9: Combination: intro, aesop
theorem {name}_v9 : {statement_clean} := by
  intro
  aesop
  
-- Attempt 10: Sorry (placeholder - marks as unproved)
theorem {name}_v10 : {statement_clean} := by
  sorry
"""
    return lean_content


def check_lean_file(file_path: str, timeout: int = 60) -> Tuple[bool, str, List[str]]:
    """
    Check a LEAN file for errors using the lean command.
    
    Returns:
        Tuple of (success, output, successful_tactics)
    """
    try:
        result = subprocess.run(
            ['lean', file_path],
            capture_output=True,
            text=True,
            timeout=timeout
        )
        
        output = result.stdout + result.stderr
        
        # Parse output to find which proof attempts succeeded
        successful_tactics = []
        
        # If no errors, all proofs succeeded
        if result.returncode == 0 and "error" not in output.lower():
            successful_tactics = ['all']
            return True, output, successful_tactics
        
        # Parse errors to identify which versions failed
        lines = output.split('\n')
        failed_versions = set()
        for line in lines:
            if 'error' in line.lower():
                # Extract version number from error
                match = re.search(r'_v(\d+)', line)
                if match:
                    failed_versions.add(int(match.group(1)))
        
        # Determine successful tactics
        all_versions = list(range(1, 11))
        successful_versions = [v for v in all_versions if v not in failed_versions]
        
        tactic_names = {
            1: 'aesop',
            2: 'simp_all+omega',
            3: 'decide',
            4: 'tauto',
            5: 'simp+omega',
            6: 'trivial',
            7: 'rfl',
            8: 'intro+simp_all+aesop',
            9: 'intro+aesop',
            10: 'sorry'
        }
        
        successful_tactics = [tactic_names.get(v, f'v{v}') for v in successful_versions]
        
        success = len(successful_tactics) > 0 and 'sorry' not in successful_tactics
        
        return success, output, successful_tactics
        
    except subprocess.TimeoutExpired:
        return False, "Timeout: LEAN took too long to check the file", []
    except Exception as e:
        return False, f"Error: {str(e)}", []

In [26]:
def clean_lean_statement(statement: str) -> str:
    """
    Clean up the LEAN statement by removing any 'theorem' keywords that shouldn't be there.
    """
    # Remove 'theorem theorem_name :' pattern if present
    import re
    
    # Remove leading 'theorem' keyword and theorem name if present
    cleaned = re.sub(r'^\s*theorem\s+\w+\s*:', '', statement).strip()
    
    # Also try to remove just the theorem keyword
    if cleaned == statement:
        cleaned = re.sub(r'^\s*theorem\s+', '', statement).strip()
    
    return cleaned if cleaned else statement


def generate_simple_lean_proof(statement: str, name: str, kind: str) -> str:
    """
    Generate a LEAN file with just one powerful auto-prover attempt.
    This is simpler and faster - just use aesop which handles most cases.
    """
    # Clean the statement first
    statement_clean = clean_lean_statement(statement.strip())
    
    # Add proper LEAN 4 file header with imports
    lean_content = f"""-- LEAN 4 file for theorem: {name}
-- Original HOL4 {kind}

import Mathlib.Tactic
import Mathlib.Data.List.Basic
import Mathlib.Data.Option.Basic

-- Theorem with auto-prover
theorem {name} : {statement_clean} := by
  aesop
"""
    return lean_content


def attempt_simple_proof(statement: str, name: str, kind: str, project_dir: str) -> Dict:
    """
    Attempt to prove using just aesop (the simplest approach).
    
    Returns:
        Dictionary with proof results
    """
    # Create LEAN file
    lean_file_path = Path(project_dir) / "ProofProject" / f"{name}.lean"
    
    # Generate LEAN proof with just aesop
    lean_content = generate_simple_lean_proof(statement, name, kind)
    
    # Write to file
    lean_file_path.write_text(lean_content)
    
    # Check the file
    try:
        result = subprocess.run(
            ['lean', str(lean_file_path)],
            capture_output=True,
            text=True,
            timeout=60
        )
        
        output = result.stdout + result.stderr
        success = result.returncode == 0 and "error" not in output.lower()
        
        return {
            "name": name,
            "kind": kind,
            "statement": statement,
            "cleaned_statement": clean_lean_statement(statement),
            "proved": success,
            "tactic": "aesop" if success else "failed",
            "lean_output": output[:1000] if output else "",  # Show more output for debugging
            "lean_file": str(lean_file_path)
        }
        
    except subprocess.TimeoutExpired:
        return {
            "name": name,
            "kind": kind,
            "statement": statement,
            "cleaned_statement": clean_lean_statement(statement),
            "proved": False,
            "tactic": "timeout",
            "lean_output": "Timeout after 60 seconds",
            "lean_file": str(lean_file_path)
        }
    except Exception as e:
        return {
            "name": name,
            "kind": kind,
            "statement": statement,
            "cleaned_statement": clean_lean_statement(statement),
            "proved": False,
            "tactic": "error",
            "lean_output": str(e),
            "lean_file": str(lean_file_path)
        }

## Main Proof Attempt Function

## Simple Auto-Proof Approach (Recommended)

Instead of trying multiple tactics, we can use just **`aesop`** - LEAN's most powerful automated reasoner. This is simpler, faster, and works for most cases.

In [27]:
def prove_all_statements_simple(input_file: str, output_file: str, project_dir: str) -> None:
    """
    Attempt to prove all statements using just aesop (simpler and faster).
    """
    # Load translated statements
    print(f"Loading translated statements from: {input_file}")
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print(f"Found {len(data)} statements to prove")
    print("Using aesop auto-prover...\n")
    
    # Create LEAN project
    if not create_lean_project(project_dir):
        print("Failed to create LEAN project. Aborting.")
        return
    
    # Attempt to prove each statement
    results = []
    proved_count = 0
    
    for i, item in enumerate(data):
        print(f"Proving {i+1}/{len(data)}: {item['name']}... ", end='', flush=True)
        
        result = attempt_simple_proof(
            item['statement'],
            item['name'],
            item['kind'],
            project_dir
        )
        
        # Add original HOL4 for reference
        if 'original_hol4' in item:
            result['original_hol4'] = item['original_hol4']
        
        results.append(result)
        
        if result['proved']:
            proved_count += 1
            print(f"✓ PROVED")
        else:
            print(f"✗ FAILED ({result['tactic']})")
    
    # Save results
    print(f"\n\nSaving results to: {output_file}")
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"PROOF SUMMARY")
    print(f"{'='*60}")
    print(f"Total statements: {len(data)}")
    print(f"Successfully proved: {proved_count}")
    print(f"Failed to prove: {len(data) - proved_count}")
    print(f"Success rate: {proved_count/len(data)*100:.1f}%")
    print(f"{'='*60}")

In [28]:
def attempt_proof(statement: str, name: str, kind: str, project_dir: str) -> Dict:
    """
    Attempt to prove a LEAN statement using various tactics.
    
    Returns:
        Dictionary with proof results
    """
    # Create temporary LEAN file
    lean_file_path = Path(project_dir) / "ProofProject" / f"{name}.lean"
    
    # Generate LEAN proof attempts
    lean_content = generate_lean_proof_file(statement, name, kind)
    
    # Write to file
    lean_file_path.write_text(lean_content)
    
    # Check the file
    success, output, successful_tactics = check_lean_file(str(lean_file_path))
    
    result = {
        "name": name,
        "kind": kind,
        "statement": statement,
        "proved": success,
        "successful_tactics": successful_tactics,
        "lean_output": output[:500] if output else "",  # Truncate for brevity
        "lean_file": str(lean_file_path)
    }
    
    return result


def prove_all_statements(input_file: str, output_file: str, project_dir: str) -> None:
    """
    Attempt to prove all statements from the input JSON file.
    """
    # Load translated statements
    print(f"Loading translated statements from: {input_file}")
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print(f"Found {len(data)} statements to prove")
    
    # Create LEAN project
    if not create_lean_project(project_dir):
        print("Failed to create LEAN project. Aborting.")
        return
    
    # Attempt to prove each statement
    results = []
    proved_count = 0
    
    for i, item in enumerate(data):
        print(f"\nProving {i+1}/{len(data)}: {item['name']}... ", end='', flush=True)
        
        result = attempt_proof(
            item['statement'],
            item['name'],
            item['kind'],
            project_dir
        )
        
        # Add original HOL4 for reference
        if 'original_hol4' in item:
            result['original_hol4'] = item['original_hol4']
        
        results.append(result)
        
        if result['proved']:
            proved_count += 1
            tactics_str = ', '.join(result['successful_tactics'])
            print(f"✓ PROVED (tactics: {tactics_str})")
        else:
            print("✗ FAILED")
    
    # Save results
    print(f"\n\nSaving results to: {output_file}")
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    # Print summary
    print(f"\n{'='*60}")
    print(f"PROOF SUMMARY")
    print(f"{'='*60}")
    print(f"Total statements: {len(data)}")
    print(f"Successfully proved: {proved_count}")
    print(f"Failed to prove: {len(data) - proved_count}")
    print(f"Success rate: {proved_count/len(data)*100:.1f}%")
    print(f"{'='*60}")

## Run Proof Attempts

**Note:** This requires LEAN 4 to be installed on your system.

Choose one of two approaches:
1. **Simple (Recommended)**: Use only `aesop` - faster and simpler
2. **Comprehensive**: Try multiple tactics to see which ones work best

In [29]:
# Choose your approach:
USE_SIMPLE_APPROACH = True  # Set to False to try multiple tactics

if lean_available:
    if USE_SIMPLE_APPROACH:
        print("Using simple approach: aesop only\n")
        prove_all_statements_simple(INPUT_FILE, OUTPUT_FILE, LEAN_PROJECT_DIR)
    else:
        print("Using comprehensive approach: multiple tactics\n")
        prove_all_statements(INPUT_FILE, OUTPUT_FILE, LEAN_PROJECT_DIR)
else:
    print("\nLEAN 4 is not available. Please install it first.")
    print("Installation instructions: https://leanprover.github.io/lean4/doc/quickstart.html")

Using simple approach: aesop only

Loading translated statements from: test_file/output_namespacePropsScript_lean.json
Found 56 statements to prove
Using aesop auto-prover...

Project directory 'lean_proof_project' already exists.
Proving 1/56: mk_id_surj... ✗ FAILED (failed)
Proving 2/56: mk_id_thm... ✗ FAILED (failed)
Proving 2/56: mk_id_thm... ✗ FAILED (failed)
Proving 3/56: nsSub_mono2... ✗ FAILED (failed)
Proving 3/56: nsSub_mono2... ✗ FAILED (failed)
Proving 4/56: nsLookup_Bind_v_some... ✗ FAILED (failed)
Proving 4/56: nsLookup_Bind_v_some... ✗ FAILED (failed)
Proving 5/56: nsLookup_to_nsLookupMod... ✗ FAILED (failed)
Proving 5/56: nsLookup_to_nsLookupMod... ✗ FAILED (failed)
Proving 6/56: nsLookup_alist_to_ns_some... ✗ FAILED (failed)
Proving 6/56: nsLookup_alist_to_ns_some... ✗ FAILED (failed)
Proving 7/56: nsLookup_alist_to_ns_none... ✗ FAILED (failed)
Proving 7/56: nsLookup_alist_to_ns_none... ✗ FAILED (failed)
Proving 8/56: nsLookup_nsLift... ✗ FAILED (failed)
Proving 8/56: 

## Analyze Proof Results

In [30]:
# Load and analyze the results
with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
    results = json.load(f)

# Statistics by proof status
proved = [r for r in results if r['proved']]
failed = [r for r in results if not r['proved']]

print(f"Total theorems: {len(results)}")
print(f"Proved: {len(proved)} ({len(proved)/len(results)*100:.1f}%)")
print(f"Failed: {len(failed)} ({len(failed)/len(results)*100:.1f}%)")

# Analyze which tactics were most successful (if multiple tactics were used)
if proved and 'successful_tactics' in proved[0]:
    print("\n" + "="*60)
    print("TACTIC SUCCESS RATE")
    print("="*60)
    
    tactic_counts = {}
    for result in proved:
        for tactic in result.get('successful_tactics', []):
            if tactic != 'all' and tactic != 'sorry':
                tactic_counts[tactic] = tactic_counts.get(tactic, 0) + 1
    
    # Sort by count
    sorted_tactics = sorted(tactic_counts.items(), key=lambda x: x[1], reverse=True)
    for tactic, count in sorted_tactics:
        print(f"  {tactic}: {count} theorems")
elif proved:
    print(f"\nAll proofs used: aesop")

Total theorems: 56
Proved: 0 (0.0%)
Failed: 56 (100.0%)


## Show Sample Proofs

In [31]:
# Display first 3 successfully proved theorems
print("\n" + "="*80)
print("SAMPLE SUCCESSFUL PROOFS")
print("="*80)

for i, result in enumerate(proved[:3]):
    print(f"\n{i+1}. {result['kind']}: {result['name']}")
    print(f"   Statement: {result['statement']}")
    
    # Handle both result formats
    if 'successful_tactics' in result:
        print(f"   Successful tactics: {', '.join(result['successful_tactics'])}")
    else:
        print(f"   Tactic: {result.get('tactic', 'unknown')}")
    print("-"*80)

# Display first 3 failed proofs
if failed:
    print("\n" + "="*80)
    print("SAMPLE FAILED PROOFS")
    print("="*80)
    
    for i, result in enumerate(failed[:3]):
        print(f"\n{i+1}. {result['kind']}: {result['name']}")
        print(f"   Statement: {result['statement']}")
        print(f"   Error (truncated): {result['lean_output'][:200]}...")
        print("-"*80)


SAMPLE SUCCESSFUL PROOFS

SAMPLE FAILED PROOFS

1. Theorem: mk_id_surj
   Statement: theorem mk_id_surj {IdType PType NType : Type} (mk_id : PType → NType → IdType) : ∀ (id : IdType), ∃ (p : PType) (n : NType), id = mk_id p n
   Error (truncated): lean_proof_project\ProofProject\mk_id_surj.lean:4:0: error: unknown module prefix 'Mathlib'

No directory 'Mathlib' or file 'Mathlib.olean' in the search path entries:
c:\Users\my-pc\.elan\toolchains\...
--------------------------------------------------------------------------------

2. Theorem: mk_id_thm
   Statement: theorem mk_id_thm {IdType ModsType NType : Type} (mk_id : ModsType → NType → IdType) (id_to_mods : IdType → ModsType) (id_to_n : IdType → NType) : ∀ (id : IdType), mk_id (id_to_mods id) (id_to_n id) = id
   Error (truncated): lean_proof_project\ProofProject\mk_id_thm.lean:4:0: error: unknown module prefix 'Mathlib'

No directory 'Mathlib' or file 'Mathlib.olean' in the search path entries:
c:\Users\my-pc\.elan\toolchains\l...

## Export Proved Theorems to LEAN File

In [32]:
def export_proved_theorems(results: List[Dict], output_file: str) -> None:
    """
    Export all successfully proved theorems to a single LEAN file.
    """
    # Filter out failed proofs and 'sorry' proofs
    proved_theorems = [r for r in results if r['proved']]
    
    if 'successful_tactics' in proved_theorems[0]:
        # Filter out 'sorry' from comprehensive approach
        proved_theorems = [r for r in proved_theorems if 'sorry' not in r.get('successful_tactics', [])]
    
    lean_content = """-- Auto-proved theorems from HOL4 translation
-- Generated automatically using LEAN 4 automated provers

"""
    
    for result in proved_theorems:
        # Determine which tactic to use
        if 'successful_tactics' in result:
            # Use the first successful tactic (usually the simplest)
            best_tactic = result['successful_tactics'][0] if result['successful_tactics'] else 'sorry'
        else:
            # Simple approach - use aesop
            best_tactic = result.get('tactic', 'aesop')
        
        lean_content += f"""-- {result['kind']}: {result['name']}
theorem {result['name']} : {result['statement']} := by
  {best_tactic}

"""
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(lean_content)
    
    print(f"Exported {len(proved_theorems)} proved theorems to: {output_file}")

# Export proved theorems
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
        results = json.load(f)
    
    export_proved_theorems(results, "test_file/proved_theorems.lean")

IndexError: list index out of range